In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# 1. Load base model (tanpa top)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# 2. Bekukan semua layer sementara
base_model.trainable = False

In [ ]:
# 3. Tambahkan classifier baru di atas model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Untuk klasifikasi 3 class
])

In [ ]:
# 4. Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

In [ ]:
# 5. Preprocessing dan Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
# Set up directory for your dataset
train_dir = '/content/drive/MyDrive/transfer-learning/fast-food-dataset/train'
val_dir = '/content/drive/MyDrive/transfer-learning/fast-food-dataset/test'

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1200 images belonging to 3 classes.
Found 120 images belonging to 3 classes.


In [ ]:
# 6. Training awal (feature extractor)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=3,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.4586 - loss: 1.2720 

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


37/37 ━━━━━━━━━━━━━━━━━━━━ 784s 21s/step - accuracy: 0.4610 - loss: 1.2663 - val_accuracy: 0.5938 - val_loss: 0.8808
Epoch 2/3
 1/37 ━━━━━━━━━━━━━━━━━━━━ 11:08 19s/step - accuracy: 0.6562 - loss: 0.8007

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


37/37 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.6562 - loss: 0.8007 - val_accuracy: 0.5729 - val_loss: 0.8877
Epoch 3/3
37/37 ━━━━━━━━━━━━━━━━━━━━ 741s 20s/step - accuracy: 0.6955 - loss: 0.6860 - val_accuracy: 0.6146 - val_loss: 0.9239


In [ ]:
# 7. Fine-tuning: buka beberapa layer terakhir dari base_model
base_model.trainable = True
# Hanya fine-tune layer tertentu (misalnya 4 layer terakhir)
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [ ]:
# 8. Compile ulang dengan learning rate lebih rendah
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

In [ ]:
# 9. Fine-tuning
fine_tune_history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)

Epoch 1/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 909s 24s/step - accuracy: 0.7822 - loss: 0.5495 - val_accuracy: 0.6667 - val_loss: 0.7687
Epoch 2/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 893s 24s/step - accuracy: 0.8157 - loss: 0.4492 - val_accuracy: 0.6917 - val_loss: 0.7268
Epoch 3/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 923s 24s/step - accuracy: 0.8415 - loss: 0.3982 - val_accuracy: 0.6500 - val_loss: 0.7276
Epoch 4/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 934s 24s/step - accuracy: 0.8637 - loss: 0.3513 - val_accuracy: 0.7333 - val_loss: 0.6845
Epoch 5/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 905s 24s/step - accuracy: 0.8820 - loss: 0.3197 - val_accuracy: 0.7083 - val_loss: 0.7119


In [ ]:
# 7. Save the model (optional)
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Setelah training selesai
model.save('/content/drive/MyDrive/transfer-learning/Model/finetuning_vgg16.h5')


# 8. Evaluate the model (optional)
test_loss, test_accuracy = model.evaluate(val_generator)
print(f'Test accuracy: {test_accuracy:.4f}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
4/4 ━━━━━━━━━━━━━━━━━━━━ 71s 17s/step - accuracy: 0.6856 - loss: 0.6483
Test accuracy: 0.6750
